In [1]:
import pandas as pd
import requests
import time

In [2]:
def get_team_data(home_away, team_data):

    #replace null object with "null" string
    team_dict = eval(team_data.loc[home_away].to_json().replace("null", "'null'"))

    #player stats
    team_info = pd.DataFrame.from_dict(team_dict["data"])

    #storage dataframe
    player_stats = pd.DataFrame()

    #loop through dicts in lineups col and convert dicts to dataframes
    for player in team_info["lineups"]:
        p_stat = pd.DataFrame([player]).rename(columns={"id": "player_id"})
        player_stats = pd.concat([player_stats, p_stat], axis=0)

    #reset indexes
    team_info = team_info.reset_index(drop=True)
    player_stats = player_stats.reset_index(drop=True)

    #side-by-side join of info (team id, name, abbr, score)
    return_df = pd.concat([team_info, player_stats], axis=1)

    #rename colums
    return_df = return_df.rename(columns={"id": f"{home_away}_id",
                                          "name": f"{home_away}_name",
                                          "abbreviation": f"{home_away}_abbreviation",
                                          "score": f"{home_away}_score"})

    #drop lineups column
    return_df = return_df.drop(["lineups"], axis=1)

    return return_df



def get_match_data(data):
    df = pd.DataFrame.from_dict(data)

    home_data = get_team_data("home_team", df)
    away_data = get_team_data("away_team", df)

    # get the columns with the match information
    match_info = df.loc[["id", "status", "ot", "started_at"]].transpose()
    match_info = match_info.rename(columns={"id": "match_id"})

    #number of times to duplicate is max number of players on a team (usually 14)
    dupl_times = max([home_data.shape[0], away_data.shape[0]])

    #duplicate the match info rows as many times as there is data
    match_info = pd.concat([match_info] * dupl_times, ignore_index=True)

    #concatenate the match_info, home_data, and away_data tables together
    match_data = pd.concat([match_info, home_data, away_data], axis=1)

    return match_data

In [3]:
#urls for round 1 matches
match_urls = [4806, 4807, 4808, 4809, 4810, 4811, 4812]

match_dataframes = []

In [4]:
for match in match_urls:
    url = f"https://fantaking-api.dunkest.com/api/v1/matches/{match}/lineups"

    headers = {
        "User-Agent": "Mozilla/5.0",
        "Accept": "application/json"
    }

    # Send the request
    response = requests.get(url, headers=headers)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()  # Assuming it's in JSON format
        match_dataframes.append(get_match_data(data))
        print("done")

        #wait 5s for api reasons
        time.sleep(5)

    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")


done
done
done
done
done
done
done


In [10]:
pd.concat(match_dataframes).to_csv("match_dataframes.csv")